In [ ]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

import math
import warnings
import itertools
warnings.simplefilter("ignore", UserWarning)

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm

# Import data and show the last 10 observations
df = pd.read_excel("Welch_Goyal_PredictorData2023.xlsx", sheet_name='Quarterly')
df.tail(10)

# equity premium, i.e., the total rate of return on the stock market minus the prevailing short-term interest rate.
df['premium'] = df['CRSP_SPvw'] - df['Rfree']

# Dividend Price Ratio (dp) is the difference between the log of dividends and the log of prices. 
df['dp'] = np.log(df['D12']) - np.log(df['Index'])

# Dividend Yield (dy) is the difference between the log of dividends and the log of lagged prices.
df['dy'] = np.log(df['D12'])- np.log(df['Index'].shift(1))

# Earnings Price Ratio (ep) is the difference between the log of earnings and the log of prices.
df['ep'] = np.log(df['E12']) - np.log(df['Index'])

# Term Spread (tms) is the difference between the long term yield on government bonds and the T-bill.
df['tms'] = df['lty'] - df['tbl']

# Default Yield Spread (dfy): is the difference between BAA- and AAA- rated cor- porate bond yields.
df['dfy'] = df['BAA'] - df['AAA']

# Rename 'b/m' to 'bm'
df = df.rename(columns={'b/m':'bm'})

# List of predictors names
x_names = ['dp', 'dy', 'ep','bm', 'ntis', 'tbl', 'ltr', 'tms', 'dfy', 'infl', 'ik']


# Create a 'date' column from 'year' and 'quarter'
df['year'] = df['yyyyq'].astype(str).str[:4].astype(int)  # Extract the year
df['quarter'] = df['yyyyq'].astype(str).str[4].astype(int)  # Extract the quarter
df['date'] = pd.PeriodIndex(year=df['year'], quarter=df['quarter'], freq='Q')

df_y = df.copy()
df_y.set_index('date', inplace=True)
df_y
# Convert the PeriodIndex to DatetimeIndex
stock_return = df_y.loc['1970Q1':, 'premium']
stock_return.index = stock_return.index.to_timestamp()  # Convert PeriodIndex to DatetimeIndex
stock_return
# Plotting the actual stock return values
with sns.color_palette('deep'): #seaborn의 색상팔레트 적용
        
    plt.figure(figsize=(10, 6))
    plt.plot(stock_return.index, stock_return, label='Premium', marker='o')
    
    # Adding labels and title
    plt.xlabel('Date')
    plt.ylabel('Premium')
    plt.title('US Quarterly Stock Returns')
    plt.legend()

    # Display the plot
    plt.show()
features = df[['premium'] + x_names]
corr = features.corr()
# Generate a mask for the upper triangle
#대각선 아래는 false, 위에는 true인 corr과 크기가 동일한 행렬 생성
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(10, 8))

# Generate a custom diverging colormap
#데이터의 중심을 기준으로 두 가지 색상이 갈라지는 특성
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.show()

#Save the plot as a PNG file
plt.savefig("correlation_heatmap.png", format="png", dpi=300)
# Lag the predictors and drop rows if any of the variables has null values
# 예측 변수의 시차를 적용해 프리미엄 예측 시 이전 시점의 변수들을 사용하는 형태
df_clean = pd.concat([df[['date', 'premium']], df[x_names].shift(1)], axis=1 )
#시프트로 인해 첫번재 행이 nan이므로 이 행(axis=0)을 삭제
df_clean = df_clean.dropna(axis=0)
df_clean

current_date = '1969Q4'
forecast_date = '1970Q1'

#2023년 3분기까지의 데이터 중 x_names의 리스트에 퐣ㅁ된 모든 변수를 포함한 훈련 데이터의 특성행렬을 생성
X_train = df_clean.loc[df_clean['date'] <= current_date, x_names]
y_train = df_clean.loc[df_clean['date'] <= current_date, 'premium']

#2023년 4분기의 test데이터 생성
X_test = df_clean.loc[df_clean['date'] == forecast_date, x_names]
y_test = df_clean.loc[df_clean['date'] == forecast_date, 'premium']
# Initialize best squared error to a very large number
best_squared_error = np.inf  
#현재까지 찾은 제곱 오차 무한대로 초기화. 오차는 무조건 무한대보다 작아지므로 초기값으로 적합.
best_model = None
best_selected_features = None

# Iterate over all combinations of predictors
for i in range(1, len(x_names) + 1):
    for combo in itertools.combinations(x_names, i):
        #모든 예측변수의 조합을 시도하여 최적의 모델 find
        # Train model for the current combination of features
        X_train_selected = sm.add_constant(X_train[list(combo)])
        #선택된 예측 변수를 포함하는 훈련 데이터에 상수항을 추가(OLS 회귀모델이 절편을 포함시키게)
        model = sm.OLS(y_train, X_train_selected).fit()
        
        # Generate out-of-sample forecast for 2023Q4
        # Manually add the constant column (intercept) because X_test has one row
        X_test_selected = X_test[list(combo)].copy()
        X_test_selected.insert(0, 'const', 1)
    
        # forecast and squared forecast error
        forecast = model.predict(X_test_selected)
        squared_error = (y_test - forecast)**2

        # Check if this is the best model based on the squared forecast error
        if squared_error.iloc[0] < best_squared_error:
            #현재 조합의 제곱 오차가 이전까지의 최적오차보다 작은 지 확인
            best_squared_error = squared_error.iloc[0]
            #테스트 세트가 단일 행이므로 첫 번째 요소 가져옴
            best_model = model
            best_selected_x = list(combo)
            
            
            
print(f"Selected predictors: {best_selected_x}")#최적의 예측변수 조합 프린트
print(f"Actual stock return value: {y_test.iloc[0].round(4)} and the forecast {forecast.iloc[0].round(4)}")
print(f"Squared forecast error: {best_squared_error}")

In [2]:
import pandas as pd
import numpy as np
import itertools
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter("ignore", UserWarning)

# 필요한 라이브러리 임포트
# ...

# 데이터 불러오기 및 전처리 (이전 코드와 동일)
df = pd.read_excel("Welch_Goyal_PredictorData2023.xlsx", sheet_name='Quarterly')

# 프리미엄 및 예측 변수 생성
df['premium'] = df['CRSP_SPvw'] - df['Rfree']
df['dp'] = np.log(df['D12']) - np.log(df['Index'])
df['dy'] = np.log(df['D12']) - np.log(df['Index'].shift(1))
df['ep'] = np.log(df['E12']) - np.log(df['Index'])
df['tms'] = df['lty'] - df['tbl']
df['dfy'] = df['BAA'] - df['AAA']
df = df.rename(columns={'b/m':'bm'})
x_names = ['dp', 'dy', 'ep','bm', 'ntis', 'tbl', 'ltr', 'tms', 'dfy', 'infl', 'ik']

# 날짜 컬럼 생성
df['year'] = df['yyyyq'].astype(str).str[:4].astype(int)
df['quarter'] = df['yyyyq'].astype(str).str[4].astype(int)
df['date'] = pd.PeriodIndex(year=df['year'], quarter=df['quarter'], freq='Q')

# 예측 변수 시차 적용 및 결측치 제거
df_clean = pd.concat([df[['date', 'premium']], df[x_names].shift(1)], axis=1 )
df_clean = df_clean.dropna(axis=0)
df_clean = df_clean.reset_index(drop=True)

# 예측 결과를 저장할 데이터프레임 생성
results = df_clean[['date', 'premium']].copy()
results = results[results['date'] >= '1970Q1']  # 1970Q1부터 예측 시작
results = results.reset_index(drop=True)

# 예측값을 저장할 리스트 초기화
forecast_values = []
benchmark_forecast_values = []

# 모든 시점에 대해 재귀적으로 모델 추정 및 예측
for i in range(len(results)):
    forecast_date = results.loc[i, 'date']
    
    # 훈련 데이터 설정: 현재 예측 시점 이전의 모든 데이터
    train_data = df_clean[df_clean['date'] < forecast_date]
    X_train = train_data[x_names]
    y_train = train_data['premium']
    
    # 테스트 데이터 설정: 현재 예측 시점의 데이터
    X_test = df_clean[df_clean['date'] == forecast_date][x_names]
    y_test = df_clean[df_clean['date'] == forecast_date]['premium'].values[0]
    
    # 상수항 추가
    X_train_const = sm.add_constant(X_train)
    X_test_const = sm.add_constant(X_test)
    
    # OLS 회귀 모델 추정
    model = sm.OLS(y_train, X_train_const).fit()
    
    # 예측 수행
    forecast = model.predict(X_test_const)
    forecast_values.append(forecast.values[0])
    
    # 벤치마크 모델 예측: 훈련 데이터의 프리미엄 평균 사용
    mean_premium = y_train.mean()
    benchmark_forecast_values.append(mean_premium)
    
    # 진행 상황 출력 (선택 사항)
    print(f"Forecast for {forecast_date}: {forecast.values[0]:.4f}, Actual: {y_test:.4f}")

# 결과 데이터프레임에 예측값 추가
results['Forecast'] = forecast_values
results['Benchmark'] = benchmark_forecast_values

# 실제값과 예측값 비교
print(results.head())

# 외부표본 MSE 계산
mse_model = np.mean((results['premium'] - results['Forecast'])**2)
mse_benchmark = np.mean((results['premium'] - results['Benchmark'])**2)

print(f"Out-of-sample MSE of the model: {mse_model:.6f}")
print(f"Out-of-sample MSE of the benchmark model: {mse_benchmark:.6f}")

# 예측값과 실제값, 벤치마크 모델 예측값을 함께 플롯
plt.figure(figsize=(14, 7))
plt.plot(results['date'].to_timestamp(), results['premium'], label='Actual', marker='o')
plt.plot(results['date'].to_timestamp(), results['Forecast'], label='Forecast', marker='o')
plt.plot(results['date'].to_timestamp(), results['Benchmark'], label='Benchmark', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Premium')
plt.title('Actual vs. Forecasted Stock Returns')
plt.legend()
plt.show()


ValueError: shapes (1,11) and (12,) not aligned: 11 (dim 1) != 12 (dim 0)